In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import pandas as pd
import numpy as np
import torch
import datetime
from tqdm import tqdm_notebook
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model

import model as M
import unet_parts
from dataset import SegmentationDataset, SegmentationInferenceDataset
from data import *
from util import *
from loss import FocalLoss
from lovasz_loss import lovasz_hinge_flat

In [3]:
torch.cuda.is_available()
device = torch.device('cuda')

# Setup Dataset

In [4]:
train_df, val_df, _ = get_dfs()

In [5]:
train_dataset = SegmentationDataset(train_df, size=(128, 128), use_depth_channels=True)
val_dataset = SegmentationInferenceDataset(val_df, input_size=(128, 128), use_depth_channels=True, with_gt=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(n_channels=3, n_classes=1)
model = model.to(device)

#criterion = FocalLoss(gamma=2.0, alpha=0.25)
#criterion = nn.BCEWithLogitsLoss()
#criterion = FocalLoss(gamma=2.0, alpha=0.25)
criterion = lovasz_hinge_flat
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:
def train(model, n_epoch, train_iter, val_iter):
    for epoch in range(n_epoch):
        model.train()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            output = model(data)
            loss = criterion(output.view(-1), target.view(-1))
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 50 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
                
        gc.collect()
        torch.cuda.empty_cache()
        
        with torch.no_grad():
            iou = evaluate(model, val_iter, device=device)
        print('[{}] Train Epoch: {}\tIoU: {:.6f}'.format(now, epoch, iou))
    return model

In [8]:
model = train(model, 20, train_loader, val_loader)

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


[2018-10-05 22:59:00.210458] Train Epoch: 0 [0/3200 (0%)]	Average loss: 0.073375
[2018-10-05 22:59:14.730294] Train Epoch: 0 [800/3200 (25%)]	Average loss: 0.068076
[2018-10-05 22:59:29.273665] Train Epoch: 0 [1600/3200 (50%)]	Average loss: 0.066185
[2018-10-05 22:59:43.833689] Train Epoch: 0 [2400/3200 (75%)]	Average loss: 0.065213
[2018-10-05 22:59:43.833689] Train Epoch: 0	IoU: 0.221125
[2018-10-05 23:00:13.127859] Train Epoch: 1 [0/3200 (0%)]	Average loss: 0.075234
[2018-10-05 23:00:27.840626] Train Epoch: 1 [800/3200 (25%)]	Average loss: 0.061272
[2018-10-05 23:00:42.587864] Train Epoch: 1 [1600/3200 (50%)]	Average loss: 0.059909
[2018-10-05 23:00:57.343933] Train Epoch: 1 [2400/3200 (75%)]	Average loss: 0.059455
[2018-10-05 23:00:57.343933] Train Epoch: 1	IoU: 0.538375
[2018-10-05 23:01:19.394453] Train Epoch: 2 [0/3200 (0%)]	Average loss: 0.053158
[2018-10-05 23:01:34.284015] Train Epoch: 2 [800/3200 (25%)]	Average loss: 0.059282
[2018-10-05 23:01:49.221881] Train Epoch: 2 [1600

In [9]:
save_model(model, 'unet_res_lovaszloss_sgd_3ch')